In [42]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize,sent_tokenize,Text,PorterStemmer,WordNetLemmatizer,pos_tag
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import networkx as nx
import math
import matplotlib.pyplot as plt
import spacy
from spellchecker import SpellChecker
import collections as clt
import time
from community import best_partition

In [13]:
df = pd.read_csv('reviews_full_v5.csv')
plt.rcParams["figure.figsize"] = (10,7) #defining size of plots
sp = spacy.load('en_core_web_sm')

In [14]:
df['Text'] = df['Text'].str.lower().str.replace("'s",'').str.replace("'",'').str.replace('-','').str.replace('(',')').str.replace(')','').str.replace(',','').str.replace('.','')
print(df['Text'].head(10))

0    i originally bought chaosphere in late 2000 an...
1    a midpoint between carcass early grind albums ...
2    im not hardcore into metal at all really but i...
3    this is not a new album but in fact five album...
4    ive been trying for months to come up with an ...
5    andromeda extension of the wish clearly one of...
6    to start off i must say that i did not always ...
7    psycroptic  the isle of disenchantment self re...
8    absolute steel  the fair bitch project edgerun...
9    purgation  realm of the dead mcdself released ...
Name: Text, dtype: object


In [37]:
B = nx.Graph()
#stemmer = PorterStemmer() #faster
#stemmer = WordNetLemmatizer() #slower but more accurate
vocab = set()
a = time.time()
spell = SpellChecker()
N_albums = len(set(df.Release))
all_stopwords = sp.Defaults.stop_words
for index, row in df.head(5000).iterrows():
    text = pos_tag(word_tokenize(row['Text']))
    album = row['Release'] + 'A'
    #print(text)
    for word,tag in text:
        #print(word)
        word = word.lower()
        if word.isalpha() and len(word) > 4 and len(word) < 15 and tag == 'JJ' and word not in all_stopwords and len(spell.unknown([word])) == 0:
            if not B.has_edge(album, word):
                B.add_edge(album, word, weight = 0)
            B[album][word]['weight'] += 1
            vocab.add(word)
print(B.number_of_nodes(), B.number_of_edges(),time.time()-a)

9569 88382 91.55017757415771


In [38]:
G = nx.Graph()
for word in vocab:
    for album in B.adj[word]:
        for word1 in B.adj[album]:
            if word < word1: #order is important to avoid repeats
                if not G.has_edge(word,word1):
                    G.add_edge(word, word1, weight = 0)
                G[word][word1]['weight'] += 1

In [9]:
'''df = pd.read_csv('reviewsv7.csv')
mp = {}
for index, row in df.iterrows():
    mp[(row['User'], row['Release'])] = (row['Text'],row['Sentiment'])

df = pd.read_csv('reviews_full_v3.csv')
text = []
senti = []
for index, row in df.iterrows():
    if (row['User'], row['Release']) in mp:
        text.append(mp[(row['User'], row['Release'])][0])
        senti.append(mp[(row['User'], row['Release'])][1])
    else:
        text.append(np.nan)
        senti.append(np.nan)
df['Text'] = text
df['Sentiment'] = senti
df.to_csv('reviews_full_v4.csv', index = False)'''